In [1]:
#Customers who left are 1, current customers are 0
#Males are 0, females are 1


#LOOK UP KURTOSIS, THE GUY DID IT ON MONTHLY TENURE


#NOT SURE IF I SHOULD ONEHOTENCODE OR JUST TOKENIZE AS 0-4 FOR CREDIT CARD TYPE


#DO A RATIO BETWEEN INCOME AND EDUCATION? AND AGE?
    #doing 1/age multiplied by education value
    #NEED TO INCORPORATE FOR EACH GENDER (Diff earning potential male/female) AND REWORK EDUCATION WEIGHTS(?)

#DO A RATIO BETWEEN DEPENDENTS AND MARRIAGE?

#DO A RATIO BETWEEN TOTAL TRANSACTION AMOUNT AND TRANSACTION COUNT?

#DO RATIO BETWEEN INCOME AND BALANCE
    #not sure if we can, income is just in a range


#Open-to-Buy: The difference between the credit limit assigned to a cardholder account and the present balance on the account.
    #DO RATIO BETWEEN THIS AND CREDIT LIMIT


#CREDIT SCORE TO AGE RATIO

In [40]:
import pandas as pd
import numpy as np
from pathlib import Path

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [3]:

starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_project/Resources/BankChurners.csv"))

starter_df = starter_df[starter_df != 'Unknown'].dropna()
starter_df.sample(5)


,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
6501,765459858,Attrited Customer,47,M,3,Graduate,Divorced,Less than $40K,Blue,40,...,6397.0,0,6397.0,0.632,2511,56,0.556,0.000,0.997370,0.002628
3368,802031583,Existing Customer,53,M,2,Uneducated,Single,$80K - $120K,Blue,47,...,22855.0,0,22855.0,0.741,3711,83,0.804,0.000,0.000034,0.999970
1765,714043833,Existing Customer,37,F,3,Graduate,Single,Less than $40K,Blue,36,...,3685.0,1396,2289.0,0.974,3016,85,0.809,0.379,0.000339,0.999660
7660,715146558,Attrited Customer,34,F,2,College,Married,Less than $40K,Blue,36,...,1696.0,0,1696.0,0.782,2325,56,0.556,0.000,0.994320,0.005680
8970,718889508,Existing Customer,46,M,2,College,Divorced,$60K - $80K,Blue,37,...,10074.0,1223,8851.0,0.955,8256,99,0.623,0.121,0.000105,0.999890


In [4]:
starter_df['Education_Level'].value_counts()

Graduate         2591
High School      1653
Uneducated       1204
College           844
Post-Graduate     431
Doctorate         358
Name: Education_Level, dtype: int64

In [5]:
#Shorten column names so they display better
replacement_column_names = {'Attrition_Flag': 'Status',
        'Customer_Age': 'Age',
        'Education_Level': 'Education',
        'Dependent_count': 'Dependents',
        'Income_Category': 'Income',
        'Card_Category': 'Card Type',
        'Months_on_book': 'Tenure(month)'
        }
starter_df.rename(columns=replacement_column_names, inplace=True)

In [6]:
#Drop unknowns
starter_df = starter_df[starter_df.Marital_Status != 'Unknown']
starter_df = starter_df[starter_df.Education != 'Unknown']
starter_df = starter_df[starter_df.Income != 'Unknown']

#NOT USING RIGHT NOWReplace M/F with -1/1
#starter_df = starter_df.replace({'Gender' : {'M': '-1', 'F': '1'}})

#Replace M/F with 0,1
starter_df = starter_df.replace({'Gender' : {'M': '0', 'F': '1'}})

#Replace customer's status to 0 if they left and 1 if they stayed
starter_df = starter_df.replace({'Status' : {'Attrited Customer': 1, 'Existing Customer': 0}})





In [7]:
#use feature creation to make a ratio between dependents and income sources
#consider someone divorced as Single. only 10% of divorced people receive alimony payments
        #either a 1 (single/divorced) or a 2 (married)
        #divide that by the amount of dependents plus one, which represents how many people you take care of
                #i.e. a single person with no dependents only takes care of themself, so they are a 1. 
starter_df = starter_df.replace({'Marital_Status' : {'Divorced': 1, 'Single': 1, 'Married' : 2}})
starter_df['MaritalDependentRatio'] = ((starter_df['Marital_Status'] / (starter_df['Dependents'] + 1)) / 2).round(2)
starter_df.drop(columns=['Marital_Status', 'Dependents'], inplace=True)

#Replace card categories from Blue/Silver/Gold/Plat to 0/1/2/3/4
starter_df = starter_df.replace({'Card Type' : {'Blue': 0, 'Silver': 1, 'Gold' : 3, 'Platinum' : 4}})


#Drop irrelevant columns
starter_df = starter_df.drop(columns='CLIENTNUM')

In [8]:
#use feature creation to make a ratio between avg open to buy and credit limit

starter_df['CreditUsage'] = (starter_df['Avg_Open_To_Buy'] / starter_df['Credit_Limit']).round(2)
starter_df.drop(columns=['Avg_Open_To_Buy', 'Credit_Limit'], inplace=True)

In [9]:
#use feature creation to make a ratio between transaction amount and trans count
starter_df['AvgTransValue'] = (starter_df['Total_Trans_Amt'] / starter_df['Total_Trans_Ct']).round(2)
starter_df.drop(columns=['Total_Trans_Amt', 'Total_Trans_Ct'], inplace=True)

In [10]:
#There education categories are vague/ambiguous 
#In 2021, the highest level of education of the population age 25 and older in the United States was distributed as follows: 
    #8.9% had less than a high school diploma or equivalent.
        #our dataset shows 17% uneducated
    #27.9% had high school graduate as their highest level of school completed. 
        #our dataset shows 23.3%
    #14.9% had completed some college but not a degree.
        #ours only has an ambiguous 'graduate' as 36.6%. could be this, could mean post-grad, term is used interchangeably.
    #10.5% had an associate degree as their highest level of school completed.
        #our dataset has no distinction between associate/undergrad. just says 'college'. ours shows 11.9% for college
    #23.5% had a bachelor’s degree as their highest degree.
        #ours shows 11.9% for college
    #14.4% had completed an advanced degree such as a master’s degree, professional degree or doctoral degree. 
        #ours shows 6.1% post-graduate, 5.1% doctorate, and the ambiguous 36.6% 'graduate'ArithmeticError


counts = starter_df['Education'].value_counts()
percs = starter_df['Education'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])

,count,percentage
Graduate,2591,36.6%
High School,1653,23.3%
Uneducated,1204,17.0%
College,844,11.9%
Post-Graduate,431,6.1%
Doctorate,358,5.1%


In [11]:
#It's hard to tell what they mean by 'graduate'. Unfortunately, its the bulk of our data, so we should not drop it.
    #The existence of a 'college' column suggests that graduate could mean beyond college. 
        #Alternatively, it could mean you attended college but did not graduate, and 'graduate' represents standard undergrads.
#The fact that this such a high amount of the data (36.6%), its likely a mix of multiple categories not included, i.e. trade schools, associate degrees, dropouts, specializations
#We should try to create a weighting system that roughly ranks them based on years at school (?)
    #Essentially:
        #if you are uneducated or just high school, you are a 0.
        #if you are college or graduate, you are a 1
        #if you are post-grad or doctorate, you are a 2

#Change education: 0 if high school or less, 1 if college or graduate, 2 if post-grad or doctorate
starter_df = starter_df.replace({'Education' : {'Uneducated': 0, 'High School': 0, 'College' : 1, 'Graduate' : 1, 'Post-Graduate' : 2, 'Doctorate' : 2}})



In [12]:
starter_df = starter_df.dropna()

In [13]:
#use OneHotEncoder for the income categories
enc = OneHotEncoder(sparse=False)
categorical_variables = ['Income']
encoded_data = enc.fit_transform(starter_df[categorical_variables])

encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)

encoded_df = pd.concat(
    [
        starter_df,
        encoded_df
    ],
    axis=1
)
encoded_df.drop(columns=['Income'], inplace=True)
encoded_df = encoded_df.dropna()


c:\python_files\Anaconda3\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
encoded_df = encoded_df.dropna()
encoded_df.sample(15)

,Status,Age,Gender,Education,Card Type,Tenure(month),Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,...,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2,MaritalDependentRatio,CreditUsage,AvgTransValue,Income_$120K +,Income_$40K - $60K,Income_$60K - $80K,Income_$80K - $120K,Income_Less than $40K
6632,0.0,47.0,1,1.0,0.0,38.0,5.0,1.0,2.0,1671.0,...,0.000057,0.999940,0.12,0.45,50.81,0.0,0.0,0.0,0.0,1.0
1102,0.0,44.0,1,1.0,0.0,24.0,5.0,1.0,4.0,1668.0,...,0.000159,0.999840,0.12,0.82,40.77,1.0,0.0,0.0,0.0,0.0
702,0.0,47.0,0,1.0,0.0,35.0,4.0,3.0,2.0,1010.0,...,0.000175,0.999830,0.33,0.72,26.51,1.0,0.0,0.0,0.0,0.0
3186,0.0,50.0,0,1.0,0.0,38.0,6.0,3.0,3.0,1153.0,...,0.000317,0.999680,0.20,0.59,56.52,0.0,0.0,0.0,0.0,1.0
146,0.0,41.0,1,1.0,0.0,32.0,6.0,3.0,2.0,2117.0,...,0.000180,0.999820,0.17,0.06,52.16,0.0,0.0,1.0,0.0,0.0
3501,1.0,42.0,0,1.0,0.0,38.0,2.0,3.0,3.0,1767.0,...,0.996830,0.003175,0.25,0.75,50.50,0.0,0.0,0.0,0.0,1.0
5939,0.0,29.0,1,1.0,0.0,25.0,6.0,3.0,2.0,1746.0,...,0.000180,0.999820,0.33,0.36,55.32,0.0,0.0,0.0,0.0,1.0
3213,0.0,49.0,0,0.0,0.0,39.0,6.0,3.0,2.0,1963.0,...,0.000175,0.999830,0.33,0.73,58.96,0.0,0.0,0.0,0.0,1.0
1316,0.0,46.0,0,1.0,0.0,36.0,5.0,2.0,2.0,807.0,...,0.000115,0.999890,0.25,0.92,48.53,1.0,0.0,0.0,0.0,0.0
3562,0.0,49.0,1,0.0,0.0,45.0,3.0,3.0,0.0,1038.0,...,0.000067,0.999930,0.25,0.37,59.29,0.0,0.0,0.0,0.0,1.0


In [15]:
scaler = StandardScaler()
encoded_df[['Tenure(month)', 'Total_Revolving_Bal']] = scaler.fit_transform(encoded_df[['Tenure(month)', 'Total_Revolving_Bal']])

In [16]:
encoded_df['Gender'].value_counts()

0    2674
1    2338
Name: Gender, dtype: int64

In [18]:
encoded_df.drop(columns=['Total_Relationship_Count', 
'Contacts_Count_12_mon', 
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', 
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1'
],
inplace=True
)

In [19]:
encoded_df

,Status,Age,Gender,Education,Card Type,Tenure(month),Months_Inactive_12_mon,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,MaritalDependentRatio,CreditUsage,AvgTransValue,Income_$120K +,Income_$40K - $60K,Income_$60K - $80K,Income_$80K - $120K,Income_Less than $40K
0,0.0,45.0,0,0.0,0.0,0.354786,1.0,-0.483128,1.335,1.625,0.061,0.25,0.94,27.24,0.0,0.0,1.0,0.0,0.0
1,0.0,49.0,1,1.0,0.0,0.959693,1.0,-0.375311,1.541,3.714,0.105,0.08,0.90,39.12,0.0,0.0,0.0,0.0,1.0
2,0.0,51.0,0,1.0,0.0,-0.008159,1.0,-1.446039,2.594,2.333,0.000,0.25,1.00,94.35,0.0,0.0,0.0,1.0,0.0
4,0.0,40.0,0,0.0,0.0,-1.822881,1.0,-1.446039,2.175,2.500,0.000,0.25,1.00,29.14,0.0,1.0,0.0,0.0,0.0
5,0.0,44.0,0,1.0,0.0,-0.008159,1.0,0.099328,1.376,0.846,0.311,0.33,0.69,45.33,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7076,0.0,38.0,1,0.0,0.0,-1.096992,2.0,0.519440,0.798,0.868,0.893,0.33,0.11,61.27,0.0,0.0,1.0,0.0,0.0
7077,0.0,42.0,1,0.0,0.0,-0.855029,3.0,-1.446039,0.770,0.860,0.000,0.25,1.00,59.24,0.0,0.0,0.0,0.0,1.0
7078,0.0,42.0,1,1.0,0.0,-1.701899,3.0,-0.076648,0.735,0.711,0.460,0.25,0.54,54.36,0.0,1.0,0.0,0.0,0.0
7079,0.0,45.0,1,1.0,0.0,-0.008159,1.0,1.673199,0.786,1.391,0.752,0.20,0.25,80.09,0.0,0.0,0.0,0.0,1.0


In [21]:
y = encoded_df['Status']
X = encoded_df.drop(columns=['Status'])

In [28]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [41]:
clf = LogisticRegression()

In [42]:
#Fit the model on the training data
clf.fit(X_train, y_train)

c:\python_files\Anaconda3\envs\dev\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [43]:
#Make predictions on the test data
predictions = clf.predict(X_test)

In [44]:
residuals = y_test - predictions
residuals

4870    0.0
103     0.0
642     1.0
1956    0.0
6990    1.0
       ... 
5918    0.0
3108    1.0
4278    0.0
422     0.0
8178    0.0
Name: Status, Length: 2156, dtype: float64

In [45]:
# Evaluate the model's performance
#DOESNT WORK FOR LINREG
accuracy = accuracy_score(predictions, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 77.88%


In [46]:
#Create a dataframe showing predictions vs test
comparison = pd.DataFrame({"Testing Data Predictions": predictions,
    "Testing Data Actual Targets": y_test})
comparison = comparison.sort_index(ascending=True)
comparison.tail(25)

,Testing Data Predictions,Testing Data Actual Targets
8504,1.0,1.0
8506,1.0,1.0
8510,1.0,1.0
8517,1.0,1.0
8518,1.0,1.0
8521,1.0,1.0
8523,0.0,1.0
8539,0.0,1.0
8540,0.0,1.0
8544,1.0,1.0
